In [80]:
def generate_OneBugCorpus(FilePath, WordList):
    
    for i in range(len(WordList)):
        WordList[i][2] = 0
    
    with open(FilePath) as TextFile:
        BugFile = TextFile.read().split(' ')    


    BugCorpus = [] # this file delete word that is not in the word list; 

    for i in range(len(BugFile)):
        flag = False
        for j in range(len(WordList)):
            if BugFile[i] == WordList[j][0]:
                flag = True

                if WordList[j][2] == 0:
                    BugCorpus.append(BugFile[i])

                WordList[j][2] = WordList[j][2] + 1
                break        
       
    
    
    return BugCorpus

In [81]:
# this process intends to save all file names in an array; 

import os

path = 'C:\\Users\\take_\\OneDrive\\liming\\where\\BugLocator\\tmp\\BugCorpus\\'

BugFiles = []
# r=root, d=directories, f = bug files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            BugFiles.append(os.path.join(r, file))

print(len(BugFiles))

98


In [ ]:

## ----------------------------------- read word list from Wordlist.txt --------------------------------------------
with open("WordList.txt") as TextFile:
    WordList = [line.split() for line in TextFile]
    
for i in range(len(WordList)):
    WordList[i].append(0) # this is the 3rd column, indicating frequency of this word in this document;
    WordList[i].append(0) # this is the 4th column, indicating the id of this word; 
    
## ----------------------------------- read bug file 100040.txt ------------------------------------------------    
#ThisBugCorpus = generate_OneBugCorpus(BugFiles[0], WordList)
#print(len(ThisBugCorpus))
#print(ThisBugCorpus)

BugCorpuses = []
for i in range(len(BugFiles)):
    ThisBugCorpus = generate_OneBugCorpus(BugFiles[i], WordList)
    ThisBugCorpus.insert(0, BugFiles[i])
    BugCorpuses.append(ThisBugCorpus)
    
for i in range(5):
    print(BugCorpuses[i])

In [73]:

with open("CodeCorpus.txt") as TextFile:
    CodeList = TextFile.read().split()   

#print(len(CodeList))
CodeFiles = []
ThisCodeFile = []

for i in range(len(CodeList)):
    if CodeList[i][-5:] == ".java":        
        CodeFiles.append(ThisCodeFile)
        ThisCodeFile = []
        ThisCodeFile.append(CodeList[i])
    else:
        ThisCodeFile.append(CodeList[i])

CodeFiles.pop(0) # delete the first element;         

print(len(CodeFiles))

483


In [74]:
import math

with open(BugFiles[0]) as TextFile:
    BugFile = TextFile.read().split(' ')    
    
#print(BugFile)

ThisWord = "build"
# count ftd;
ftd = 0
for i in range(len(BugFile)):
    if BugFile[i] == ThisWord:
        ftd = ftd + 1
        
print("ftd:", ftd)
#print(BugFile)
NumDocs = len(BugFiles);
print("#docs:", NumDocs)

# count nt;
nt = 0
for i in range(NumDocs):
    with open(BugFiles[i]) as TextFile:
        ThisBugFile = TextFile.read().split(' ') 
    
    flag = False    
    
    for j in range(len(ThisBugFile)):
        if ThisWord==ThisBugFile[j]:
            flag = True
            break
    
    if flag == True:
        nt = nt + 1
    
print("nt:", nt)

w = (math.log(ftd)+1) * math.log(NumDocs/nt)
print(w)

ftd: 1
#docs: 98
nt: 13
2.020018121209035


In [75]:
WordList = []

with open("BugTermList.txt") as TextFile:
    TempList = TextFile.read().split('\n') 
    
for i in range(len(TempList)):
    ThisRow = []
    ThisRow.append(TempList[i])
    ThisRow.append(0) # this is the 2nd columnn, which is left blank;
    ThisRow.append(0) # this is the 3rd column, indicating frequency of this word in this document;
    WordList.append(ThisRow)
    
    
WordList = generate_OneBugCorpus(BugFiles[2], WordList)
    

#print(WordList)
#print(TempList)

BugCorpus C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100387.txt:
10
['ctabfold', 'creat', 'mousehov', 'showtooltip', 'tooltip', 'show', 'view', 'eclips', 'hover', 'event']
